In [22]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

In [23]:
KATEGORI_ASPEK = [
    'Kamera', 'Baterai', 'Performa & Gaming', 'Layar',
    'Desain & Kualitas Bodi', 'Harga', 'Pengiriman',
    'Layanan Penjual', 'UI'
]
# Baca file hasil akhir (sudah berisi ground truth + prediksi)
df = pd.read_csv("hasil_test_akhir.csv")

In [24]:
def parse_list_safe(s):
    if not isinstance(s, str):
        return []
    s = s.strip()
    if s.startswith('[') and s.endswith(']'):
        try:
            inner = s[1:-1]
            if inner == '':
                return []
            parts = [part.strip().strip('"').strip("'") for part in inner.split(',')]
            return [p for p in parts if p]
        except:
            return []
    return []

df['prediksi_final'] = df['prediksi_akhir'].apply(parse_list_safe)

df['ground_truth'] = df.apply(
    lambda row: [k for k in KATEGORI_ASPEK if row[k] == 1],
    axis=1
)

In [25]:
def to_binary_vector(aspek_list, KATEGORI_ASPEK):
    return [1 if k in aspek_list else 0 for k in KATEGORI_ASPEK]

y_true = []
y_pred = []

for _, row in df.iterrows():
    y_true.append(to_binary_vector(row['ground_truth'], KATEGORI_ASPEK))
    y_pred.append(to_binary_vector(row['prediksi_final'], KATEGORI_ASPEK))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

In [27]:
accuracy_per_class = []
for i in range(len(KATEGORI_ASPEK)):
    acc_i = accuracy_score(y_true[:, i], y_pred[:, i])
    accuracy_per_class.append(acc_i)

label_based_accuracy = np.mean(accuracy_per_class)

macro_f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

In [28]:
print("="*50)
print("EVALUASI FINAL TEST SET")
print("="*50)
print(f"Label-based Accuracy : {label_based_accuracy:.4f}")
print(f"Macro F1-Score       : {macro_f1:.4f}")
print("="*50)

EVALUASI FINAL TEST SET
Label-based Accuracy : 0.8968
Macro F1-Score       : 0.6191


In [ ]:
# Simpan ke file
results = pd.DataFrame({
    "Metric": ["Label-based Accuracy", "Macro F1-Score"],
    "Value": [label_based_accuracy, macro_f1]
})
results.to_csv("hasil_evaluasi_final_test_set.csv", index=False)
print("Hasil disimpan ke: hasil_evaluasi_final_test_set.csv")

Hasil disimpan ke: hasil_evaluasi_final_test_set.csv
